**Imports**

In [1]:
import math
import os

import numpy as np


import pandas as pd

import sklearn as sk
import sklearn.ensemble

import pyplotterlib.standard.plotters as ppl


#
import project_path
import preproc_pipes as preProcPipeHelp

**Configuration variables**

In [2]:
TRAIN_PATH = os.path.join("..","..","raw_data","train.csv")
TEST_PATH = os.path.join("..","..","raw_data","test.csv")


#Want to restrict to recent data only
N_DAYS_USE = 400

#
VAL_PERIOD = pd.Timedelta("30d")

#
LAG_VALS = [x for x in range(16,46)]
ON_PROM_LAGS = [1,2,3,4,5,6,7,14,15,16,17,18]

#The obvious lag here is 0; since we actually always KNOW todays (and even future days)
ON_PROM_WINDOW_VALS = [2,3,4,5,6,7,14,21]

#Obvious lag is 16; this gets us a recent average sales number we can use
#7 is obviously the sensible number here...
SALE_WINDOW_VALS = [2,3,4,5,6,7,14,21] #


#
_currKwargs = {"constrainedLayout":True, "figHeightPerRow":4, "figWidthPerCol":6, "nColsGrid":4}
RECT_MULTI_PLOTTER = ppl.RectMultiPlotter(**_currKwargs)


**Import data**

In [3]:
RAW_DF = pd.read_csv(TRAIN_PATH)

In [4]:
RAW_DF.tail(3)

,id,date,store_nbr,family,sales,onpromotion
3000885,3000885,2017-08-15,9,PRODUCE,2419.729,148
3000886,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8
3000887,3000887,2017-08-15,9,SEAFOOD,16.000,0


**Add a pipeline to process the data**

In [5]:
_uniqueFams = RAW_DF["family"].unique()
_lagPromPipe = preProcPipeHelp.AddLagFeats(ON_PROM_LAGS, targCol="onpromotion")
_maPromPipe = preProcPipeHelp.AddLeftMovingAverage(ON_PROM_WINDOW_VALS, [0], targCol="onpromotion")
_maSalePipe = preProcPipeHelp.AddLeftMovingAverage(SALE_WINDOW_VALS, [16], targCol="sales")

In [6]:
_removeNDaysBehindPipe = preProcPipeHelp.RemoveDatesNDaysBehindMax(N_DAYS_USE)

_pipeComps = [ ("Add lag features", preProcPipeHelp.AddLagFeats(LAG_VALS)),
               ("Add lag onpromotion feature", _lagPromPipe ),
               ("Add transactions data", preProcPipeHelp.AddNumbTransactionsData() ),
               ("Add store wide sums", preProcPipeHelp.AddStoreWideSums() ),
               ("Add moving average sale values", _maSalePipe ),
               ("Add moving average promotion values", _maPromPipe),
               ("Remove earlier data", _removeNDaysBehindPipe ),
               ("Add a log1p target column", preProcPipeHelp.AddLog1pSales() ),
               ("Add fractional number of promotions", preProcPipeHelp.AddFractProm() ),
               ("Add oil prices", preProcPipeHelp.AddOilPriceData() ),
               ("Add basic store info", preProcPipeHelp.AddStoreInfoData()),
               ("Add ordinal encoding for store type", preProcPipeHelp.OrdEncodeStoreType() ),
               ("Add ordinal encoding for store state", preProcPipeHelp.StoreStateOrdEncode() ),
               ("Add ordinal encoding for store city", preProcPipeHelp.StoreCityOrdEncode() ),
               ("Encode family ordinally", preProcPipeHelp.EncodeFamilyArbitrary(_uniqueFams)  ),
               ("Add day of week", preProcPipeHelp.AddDayOfWeekFeat() ),
               ("Add day of month", preProcPipeHelp.AddDayOfMonth() ),
               ("Add day of year sin/cos", preProcPipeHelp.AddDayOfYearSinCos() )
             ]

preProcPipe = sk.pipeline.Pipeline(_pipeComps)

**Apply the pipeline preprocessing**

In [7]:
%%prun
#          303100908 function calls (302138458 primitive calls) in 165.842 seconds

PROC_FRAME = preProcPipe.fit_transform(RAW_DF)

/home/richard/Work/random/learning_ml/kaggle_comps/store_sales_github/shared_code/preproc_pipes.py:134: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  summedFrame = useX.groupby(["date","store_nbr"]).sum().reset_index()
/home/richard/Work/random/learning_ml/kaggle_comps/store_sales_github/shared_code/preproc_pipes.py:103: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  meanDict = transFrame.groupby(["store_nbr"]).mean()["transactions"].to_dict()
/home/richard/Work/random/learning_ml/kaggle_comps/store_sales_github/shared_code/preproc_pipes.py:134: FutureWarning: The default value of numeric_only in DataFrameGro

**Split into train and validation**

In [8]:

_maxDate = pd.to_datetime(PROC_FRAME["date"]).max() - pd.Timedelta(VAL_PERIOD)

#
TRAIN_FULL = PROC_FRAME.loc[  (pd.to_datetime(PROC_FRAME["date"]) < _maxDate) ]
VAL_FULL = PROC_FRAME.loc[ pd.to_datetime(PROC_FRAME["date"]) >= _maxDate ]

**Apply the target encoding at THIS point**

In [9]:
targEncoder =  preProcPipeHelp.TargEncodeFamilyStore()
targEncoder.fit(TRAIN_FULL)

/home/richard/Work/random/learning_ml/kaggle_comps/store_sales_github/shared_code/preproc_pipes.py:325: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  self.meanFamDict = useX.groupby("family_enc").mean()["sales"].to_dict()


In [10]:
TRAIN_FULL = targEncoder.transform(TRAIN_FULL)
VAL_FULL = targEncoder.transform(VAL_FULL)

**Create a simple wrapper function for sklearn random forest; I want to use it directly with the dataframes**

In [11]:
class RandomForestWrapper():
    
    def __init__(self, useFeats, targFeat="sales", rfKwargs=None):
        self.useFeats = useFeats
        self.targFeat = targFeat
        rfKwargs = dict() if rfKwargs is None else rfKwargs
        self.regressor = sk.ensemble.RandomForestRegressor(**rfKwargs)
    
    def fit(self, inpX, inpY=None):
        _trainX = inpX[_useFeats]
        _trainY = inpX[self.targFeat]
        self.regressor.fit(_trainX, _trainY)
        return self
    
    def predict(self, inpX):
        _useX = inpX[self.useFeats]
        return self.regressor.predict(_useX)


**Create a function to get square log error; used in the evaluation metric**

In [12]:
def addSqrLogErrorToFrame(inpFrame, predCol="pred_A", actCol="sales", outCol="sqr_log_error"):
    inpFrame[outCol] = inpFrame.apply( lambda x: (math.log(x[predCol]+1) - math.log(x[actCol]+1))**2 ,axis=1)


**Standard function for calculating various things for models with different features used**

In [13]:
#NOTE: This is for the log1p case
def fitAndScoreModel(modelName, modelFeats, targFeat="sales_log1p", rfKwargs=None):
    
    _defaultKwargs = {"max_depth":12, "max_samples":0.1}
    rfKwargs = _defaultKwargs if rfKwargs is None else rfKwargs
    
    #Fit
    ALL_MODELS[modelName] = RandomForestWrapper(modelFeats, targFeat=targFeat, rfKwargs=rfKwargs)
    ALL_MODELS[modelName].fit(TRAIN_FULL)

    #Create frames holding individual error contributions
    ALL_FRAMES_TRAIN[modelName] = TRAIN_FULL.copy()
    ALL_FRAMES_VAL[modelName] = VAL_FULL.copy()
    
    #Add predictions to frames
    ALL_FRAMES_TRAIN[modelName]["pred_A"] = np.expm1( ALL_MODELS[modelName].predict(ALL_FRAMES_TRAIN[modelName]) )
    ALL_FRAMES_VAL[modelName]["pred_A"] = np.expm1( ALL_MODELS[modelName].predict(ALL_FRAMES_VAL[modelName]) )

    #Add square errors to frames
    addSqrLogErrorToFrame(ALL_FRAMES_TRAIN[modelName])
    addSqrLogErrorToFrame(ALL_FRAMES_VAL[modelName])

    #
    ALL_SCORES_TRAIN[modelName] = math.sqrt( ALL_FRAMES_TRAIN[modelName]["sqr_log_error"].mean() )
    ALL_SCORES_VAL[modelName] = math.sqrt( ALL_FRAMES_VAL[modelName]["sqr_log_error"].mean() )

    
    

**Create a couple of different models + look at their errors**

In [14]:
ALL_MODELS = dict()
ALL_FRAMES_TRAIN = dict()
ALL_FRAMES_VAL = dict()
ALL_SCORES_TRAIN = dict()
ALL_SCORES_VAL = dict()

**Inspect frame**

In [15]:
pd.set_option("display.max_columns",90)
TRAIN_FULL

,id,date,store_nbr,family,sales,onpromotion,sales_l16,sales_l17,sales_l18,sales_l19,sales_l20,sales_l21,sales_l22,sales_l23,sales_l24,sales_l25,sales_l26,sales_l27,sales_l28,sales_l29,sales_l30,sales_l31,sales_l32,sales_l33,sales_l34,sales_l35,sales_l36,sales_l37,sales_l38,sales_l39,sales_l40,sales_l41,sales_l42,sales_l43,sales_l44,sales_l45,onpromotion_l1,onpromotion_l2,onpromotion_l3,onpromotion_l4,onpromotion_l5,onpromotion_l6,onpromotion_l7,onpromotion_l14,onpromotion_l15,onpromotion_l16,onpromotion_l17,onpromotion_l18,transactions,store_sales,store_promotions,store_sales_y,store_sales_y,sales_l16_ma2,sales_l16_ma3,sales_l16_ma4,sales_l16_ma5,sales_l16_ma6,sales_l16_ma7,sales_l16_ma14,sales_l16_ma21,onpromotion_l0_ma2,onpromotion_l0_ma3,onpromotion_l0_ma4,onpromotion_l0_ma5,onpromotion_l0_ma6,onpromotion_l0_ma7,onpromotion_l0_ma14,onpromotion_l0_ma21,sales_log1p,fract_promotions,oil_price_w10,store_city,store_state,store_type,store_cluster,store_type_ordE,store_state_ordE,store_city_ordE,family_enc,day_of_week,day_of_month,sin_day_of_year,cos_day_of_year,fam_store_mean_enc
0,2288088.0,2016-07-11,1.0,AUTOMOTIVE,4.0,0.0,5.0,4.0,1.0,2.0,5.0,3.0,0.0,4.0,1.0,3.0,6.0,5.0,9.0,3.0,10.0,2.0,4.0,5.0,3.0,4.0,2.0,9.0,4.0,1.0,3.0,5.0,6.0,4.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1712.0,9984.042999,156.0,9984.042999,329473.418983,4.5,3.333333,3.00,3.4,3.333333,2.857143,3.642857,3.857143,0.0,0.000000,0.00,0.0,0.000000,0.142857,0.285714,0.333333,1.609438,0.000000,45.884000,Quito,Pichincha,D,13,3,0,0,0,0,11,-0.177529,-0.984116,3.962060
1,2289870.0,2016-07-12,1.0,AUTOMOTIVE,1.0,0.0,2.0,5.0,4.0,1.0,2.0,5.0,3.0,0.0,4.0,1.0,3.0,6.0,5.0,9.0,3.0,10.0,2.0,4.0,5.0,3.0,4.0,2.0,9.0,4.0,1.0,3.0,5.0,6.0,4.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1648.0,9085.745000,138.0,9085.745000,299829.585000,3.5,3.666667,3.00,2.8,3.166667,3.142857,3.571429,3.857143,0.0,0.000000,0.00,0.0,0.000000,0.000000,0.214286,0.285714,0.693147,0.000000,46.040000,Quito,Pichincha,D,13,3,0,0,0,1,12,-0.194431,-0.980916,3.962060
2,2291652.0,2016-07-13,1.0,AUTOMOTIVE,1.0,0.0,5.0,2.0,5.0,4.0,1.0,2.0,5.0,3.0,0.0,4.0,1.0,3.0,6.0,5.0,9.0,3.0,10.0,2.0,4.0,5.0,3.0,4.0,2.0,9.0,4.0,1.0,3.0,5.0,6.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1792.0,13147.682000,322.0,13147.682000,433873.506000,3.5,4.000000,4.00,3.4,3.166667,3.428571,3.285714,3.904762,0.0,0.000000,0.00,0.0,0.000000,0.000000,0.214286,0.238095,0.693147,0.000000,45.872857,Quito,Pichincha,D,13,3,0,0,0,2,13,-0.211276,-0.977426,3.962060
3,2293434.0,2016-07-14,1.0,AUTOMOTIVE,4.0,0.0,5.0,5.0,2.0,5.0,4.0,1.0,2.0,5.0,3.0,0.0,4.0,1.0,3.0,6.0,5.0,9.0,3.0,10.0,2.0,4.0,5.0,3.0,4.0,2.0,9.0,4.0,1.0,3.0,5.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1719.0,9209.611000,69.0,9209.611000,303917.163000,5.0,4.000000,4.25,4.2,3.666667,3.428571,3.285714,4.000000,0.0,0.000000,0.00,0.0,0.000000,0.000000,0.214286,0.238095,1.609438,0.000000,45.843750,Quito,Pichincha,D,13,3,0,0,0,3,14,-0.228058,-0.973648,3.962060
4,2295216.0,2016-07-15,1.0,AUTOMOTIVE,8.0,0.0,8.0,5.0,5.0,2.0,5.0,4.0,1.0,2.0,5.0,3.0,0.0,4.0,1.0,3.0,6.0,5.0,9.0,3.0,10.0,2.0,4.0,5.0,3.0,4.0,2.0,9.0,4.0,1.0,3.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1673.0,12075.518000,255.0,12075.518000,398492.094000,6.5,6.000000,5.00,5.0,4.833333,4.285714,3.428571,4.142857,0.0,0.000000,0.00,0.0,0.000000,0.000000,0.142857,0.238095,2.197225,0.000000,45.743750,Quito,Pichincha,D,13,3,0,0,0,4,15,-0.244772,-0.969581,3.962060
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
712764,2938385.0,2017-07-11,54.0,SEAFOOD,3.0,1.0,3.0,3.0,1.0,0.0,9.0,4.0,4.0,3.0,3.0,2.0,0.0,3.0,0.0,5.0,2.0,4.0,1.0,5.0,0.0,4.0,3.0,4.0,3.0,0.0,0.0,7.0,1.0,7.0,5.0,6.0,1.0,0.0,2.0,1.0,0.0,1.0,0

**First baseline - using only the untransformed features in train.csv**

In [16]:
%%prun

#   


_currName = "baseline"
_useFeats = ["family_enc", "store_nbr", "store_cluster", "onpromotion"]

fitAndScoreModel(_currName, _useFeats)


**More sensible baseline, using some basic features**

In [17]:
%%prun

#          49402202 function calls (49400829 primitive calls) in 38.516 seconds


_currName = "baseline_b"
_useFeats = ["family_enc", "store_nbr", "store_cluster", "fam_store_mean_enc", "store_type_ordE",
             "store_state_ordE", "day_of_week", "onpromotion",
             "sin_day_of_year", "cos_day_of_year"]

fitAndScoreModel(_currName, _useFeats)


**Plus oil**

In [18]:
%%prun

#          

_currName = "plus_oil"
_useFeats = ["family_enc", "store_nbr", "store_cluster", "fam_store_mean_enc", "store_type_ordE",
             "store_state_ordE", "day_of_week", "onpromotion",
             "sin_day_of_year", "cos_day_of_year",  "oil_price_w10"]
_targFeat = "sales"

fitAndScoreModel(_currName, _useFeats)


**Plus extra promotions**

In [19]:
%%prun

#          

_currName = "plus_extra_prom"
_useFeats = ["family_enc", "store_nbr", "store_cluster", "fam_store_mean_enc", "store_type_ordE",
             "store_state_ordE", "day_of_week", "onpromotion",
             "sin_day_of_year", "cos_day_of_year",  "store_promotions", "fract_promotions"]
_targFeat = "sales"

fitAndScoreModel(_currName, _useFeats)


**baseline C - extra prom AND oil are there**

In [20]:
%%prun

#          


_currName = "baseline_c"
_useFeats = ["family_enc", "store_nbr", "store_cluster", "fam_store_mean_enc", "store_type_ordE",
             "store_state_ordE", "day_of_week", "onpromotion",
             "sin_day_of_year", "cos_day_of_year",  "store_promotions", "fract_promotions",
            "oil_price_w10"]
_targFeat = "sales"

fitAndScoreModel(_currName, _useFeats)


**baseline c plus some promotion moving averages**

In [21]:
%%prun

#          
_currName = "c_plus_prom_ma"
_useFeats = ["family_enc", "store_nbr", "store_cluster", "fam_store_mean_enc", "store_type_ordE",
             "store_state_ordE", "day_of_week", "onpromotion",
             "sin_day_of_year", "cos_day_of_year",  "store_promotions", "fract_promotions",
            "oil_price_w10", "onpromotion_l0_ma21", "onpromotion_l0_ma14", "onpromotion_l0_ma7"]
_targFeat = "sales"

fitAndScoreModel(_currName, _useFeats)


**baseline c plus sales moving averages**

In [22]:
%%prun

#          
_currName = "c_plus_sales_ma"
_useFeats = ["family_enc", "store_nbr", "store_cluster", "fam_store_mean_enc", "store_type_ordE",
             "store_state_ordE", "day_of_week", "onpromotion",
             "sin_day_of_year", "cos_day_of_year",  "store_promotions", "fract_promotions",
            "oil_price_w10", "sales_l16_ma7", "sales_l16_ma14", "sales_l16_ma21"]
_targFeat = "sales"

fitAndScoreModel(_currName, _useFeats)


**baseline c plus sales/prom moving average**

In [23]:
%%prun

_currName = "c_plus_sales_prom_ma"
_useFeats = ["family_enc", "store_nbr", "store_cluster", "fam_store_mean_enc", "store_type_ordE",
             "store_state_ordE", "day_of_week", "onpromotion",
             "sin_day_of_year", "cos_day_of_year",  "store_promotions", "fract_promotions",
            "oil_price_w10", "sales_l16_ma7", "sales_l16_ma14", "sales_l16_ma21",
             "onpromotion_l0_ma21", "onpromotion_l0_ma14", "onpromotion_l0_ma7"]
_targFeat = "sales"

fitAndScoreModel(_currName, _useFeats)


**baseline c plus sales 7 lag features**

In [24]:
%%prun

_currLags = ["sales_l{}".format(int(x)) for x in range(16,16+7)]

_currName = "c_plus_sales_7_lag"
_useFeats = ["family_enc", "store_nbr", "store_cluster", "fam_store_mean_enc", "store_type_ordE",
             "store_state_ordE", "day_of_week", "onpromotion",
             "sin_day_of_year", "cos_day_of_year",  "store_promotions", "fract_promotions",
            "oil_price_w10"]
_useFeats += _currLags
_targFeat = "sales"

fitAndScoreModel(_currName, _useFeats)


**baseline c plus sales 30 lag features**

In [25]:
%%prun

_currLags = ["sales_l{}".format(int(x)) for x in range(16,16+30)]

_currName = "c_plus_sales_30_lag"
_useFeats = ["family_enc", "store_nbr", "store_cluster", "fam_store_mean_enc", "store_type_ordE",
             "store_state_ordE", "day_of_week", "onpromotion",
             "sin_day_of_year", "cos_day_of_year",  "store_promotions", "fract_promotions",
            "oil_price_w10"]
_useFeats += _currLags
_targFeat = "sales"

fitAndScoreModel(_currName, _useFeats)


**baseline d**

In [26]:
%%prun

#
_currLags = ["sales_l{}".format(int(x)) for x in range(16,16+7)]

_currName = "baseline_d"

_useFeats = ["family_enc", "store_nbr", "store_cluster", "fam_store_mean_enc", "store_type_ordE",
             "store_state_ordE", "day_of_week", "onpromotion",
             "sin_day_of_year", "cos_day_of_year",  "store_promotions", "fract_promotions",
            "oil_price_w10", "sales_l16_ma7", "sales_l16_ma14", "sales_l16_ma21",
             "onpromotion_l0_ma21", "onpromotion_l0_ma14", "onpromotion_l0_ma7"]
_useFeats += _currLags
_targFeat = "sales"

fitAndScoreModel(_currName, _useFeats)


In [27]:
ALL_SCORES_TRAIN

{'baseline': 0.673876468369982,
 'baseline_b': 0.4397635049457757,
 'plus_oil': 0.4405892159169754,
 'plus_extra_prom': 0.43212411261359346,
 'baseline_c': 0.4327732805602966,
 'c_plus_prom_ma': 0.4313196928888851,
 'c_plus_sales_ma': 0.42962100311366974,
 'c_plus_sales_prom_ma': 0.4299844415480693,
 'c_plus_sales_7_lag': 0.43280166401519515,
 'c_plus_sales_30_lag': 0.43176856949419157,
 'baseline_d': 0.43134562497253287}

In [28]:
ALL_SCORES_VAL

{'baseline': 0.6590836692110084,
 'baseline_b': 0.5698796364216814,
 'plus_oil': 0.5614398107546353,
 'plus_extra_prom': 0.4648775249780584,
 'baseline_c': 0.46304670441617707,
 'c_plus_prom_ma': 0.4654689099508779,
 'c_plus_sales_ma': 0.4385125989028749,
 'c_plus_sales_prom_ma': 0.4378626046428068,
 'c_plus_sales_7_lag': 0.44257355194360803,
 'c_plus_sales_30_lag': 0.4350285304120368,
 'baseline_d': 0.43484715423951953}